In [1]:
# Standard library imports
import logging
import warnings
from pathlib import Path

# Third-party imports
import pandas as pd
import wrds

# Suppress warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore")

from settings import config
from pull_compustat import *
from pull_crsp import *
from transform_crsp import *
from transform_compustat import *
from calc_Lewellen_2014 import *

In [2]:
# Change default pandas display options

pd.options.display.max_columns = 30
pd.options.display.max_colwidth = 200
pd.set_option('display.float_format', lambda x: '%.4f' % x)
pd.set_option('display.expand_frame_repr', False)

# Global variables
RAW_DATA_DIR = Path(config("RAW_DATA_DIR"))
RAW_DATA_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR = Path(config("OUTPUT_DIR"))
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
WRDS_USERNAME = config("WRDS_USERNAME")
START_DATE = config("START_DATE")
END_DATE = config("END_DATE")


In [3]:
crsp_d = pull_CRSP_stock(
    start_date=START_DATE,
    end_date=END_DATE,
    wrds_username=WRDS_USERNAME,
    freq='D',
    data_dir=RAW_DATA_DIR,
    file_name='CRSP_stock_d.parquet',
)

Loading cached data from C:\Users\eduar\Documents\data\raw\CRSP_stock_d.parquet


In [25]:
crsp_m = pull_CRSP_stock(
    start_date=START_DATE,
    end_date=END_DATE,
    wrds_username=WRDS_USERNAME,
    freq='M',
    data_dir=RAW_DATA_DIR,
    file_name='CRSP_stock_m.parquet',
)

Loading cached data from C:\Users\eduar\Documents\data\raw\CRSP_stock_m.parquet


In [170]:
comp = pull_Compustat(
    start_date=START_DATE,
    end_date=END_DATE,
    wrds_username=WRDS_USERNAME,
    data_dir=RAW_DATA_DIR,
    file_name='Compustat_fund.parquet',
)

Loading library list...
Done
Saved data to C:\Users\eduar\Documents\data\raw\Compustat_fund.parquet


In [136]:
ccm = pull_CRSP_Comp_link_table(
    wrds_username=WRDS_USERNAME,
    data_dir=RAW_DATA_DIR,
    file_name="CRSP_Comp_Link_Table.parquet"
    )

Loading library list...
Done
Saved data to C:\Users\eduar\Documents\data\raw\CRSP_Comp_Link_Table.parquet


In [7]:
crsp_index_d = pull_CRSP_index(
    start_date=START_DATE,
    end_date=END_DATE,
    freq='D',
    wrds_username=WRDS_USERNAME,
    file_name="CRSP_index_d.parquet")

Loading cached data from C:\Users\eduar\Documents\data\raw\CRSP_index_d.parquet


In [171]:
crsp = calculate_market_equity(crsp_m)

comp = add_report_date(comp)
comp = calc_book_equity(comp)

crsp_comp = merge_CRSP_and_Compustat(crsp, comp, ccm)

crsp_comp.tail()

,permno,permco,mthcaldt,issuertype,securitytype,securitysubtype,sharetype,usincflg,primaryexch,conditionaltype,tradingstatusflg,totret,retx,prc,shrout,...,non_cash_current_assets,lct,total_debt,depreciation,dvpd,dvc,dvt,pstk,pstkl,pstkrv,txditc,seq,report_date,ps,be
230922,91405,58620,2009-06-30,CORP,EQTY,COM,NS,Y,Q,RW,A,-0.0989,-0.0989,8.1100,13225,...,<NA>,<NA>,40.4810,0.6500,<NA>,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,110.5020,2009-04-30,0.0000,110.5020
230923,91405,58620,2010-06-30,CORP,EQTY,COM,NS,Y,Q,RW,A,-0.0052,-0.0052,5.7300,13225,...,<NA>,<NA>,35.3280,0.7620,<NA>,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,107.4480,2010-04-30,0.0000,107.4480
230924,91405,58620,2011-06-30,CORP,EQTY,COM,NS,Y,Q,RW,A,0.0015,0.0015,6.7800,12797,...,<NA>,<NA>,25.1680,0.8330,<NA>,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,108.1390,2011-04-30,0.0000,108.1390
230925,91405,58620,2012-06-29,CORP,EQTY,COM,NS,Y,Q,RW,A,-0.0719,-0.0719,5.2900,12645,...,<NA>,<NA>,20.0000,0.7120,<NA>,0.6140,0.6140,0.0000,0.0000,0.0000,0.0000,107.0650,2012-04-30,0.0000,107.0650
230926,91405,58620,2013-06-28,CORP,EQTY,COM,NS,Y,Q,RW,A,0.0194,0.0194,6.3200,12645,...,<NA>,<NA>,15.0000,0.7410,<NA>,0.6690,0.6690,0.0000,0.0000,0.0000,0.0000,103.8490,2013-04-30,0.0000,103.8490


In [ ]:
subsets = get_subsets(crsp)

In [ ]:
returns = crsp.pivot_table(index='mthcaldt', columns='permno', values='retx')
    
log_size          = calc_log_size(crsp_comp)
log_bm            = calc_log_bm(crsp_comp)
return_2_12       = calc_return_12_2(crsp_comp)

In [ ]:
accruals          = calc_accruals(crsp_comp)  # or calc_accruals(crsp_comp) if you have them merged

In [156]:
roa               = calc_roa(crsp_comp)

In [157]:
log_assets_growth = calc_log_assets_growth(crsp_comp)

In [175]:
dy                = calc_dy(crsp_comp)

In [187]:
log_return_13_36  = calc_log_return_13_36(crsp_comp)

In [177]:
log_issues_12     = calc_log_issues_12(crsp_comp)

In [188]:
log_issues_36     = calc_log_issues_36(crsp_comp)

In [ ]:
betas             = calculate_rolling_beta(crsp_d, crsp_index_d)

In [190]:
std_12            = calc_std_12(crsp_d)

In [197]:
debt_price        = calc_debt_price(crsp_comp)

In [198]:
sales_price       = calc_sales_price(crsp_comp)